In [ ]:
# %pip install tensorflow
# %pip install gym
# %pip install keras
# %pip install keras-rl2

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
from matplotlib import pyplot as plt
import numpy as np 
import random
import time
import pydirectinput
import pygetwindow
import cv2
import win32gui
import pytesseract
from image_analysis_test.take_screenshot import Screenshot
from input_sending_test.input_sending import SendInput 
from number_recognition_test.number_recog import RecognizeText 
from pattern_recognition_test.pattern_recog import RecognizePattern 

In [13]:
class StepManiaEnv(Env):
    # Setup
    def __init__(self):
        super().__init__()
        self.action_space = Discrete(9)

        # Observation Array
        self.observation_space = Box(
            low=0, high=255, shape=(1, 495, 290), dtype=np.uint8)

        # Define extraction parameters for the game
        self.screenshot_helper = Screenshot()
        self.text_recog_helper = RecognizeText()
        self.input_sending_helper = SendInput()
        self.game_location = (110, 35, 400, 530)
        self.combo_location = (590, 450, 735, 480)
        self.done_location = (25, 40, 175, 70)
        pytesseract.pytesseract.tesseract_cmd = r'G:\Programme\Tesseract\tesseract.exe'

    # What is called to do something in the game
    def step(self, action):
        # Apply action
        action_map = {
            0:'no_op',
            1:'a',
            2:'d',
            3:'w',
            4:'s',
        }
        # TODO: Does this work with held notes?
        if action != 0:
            pydirectinput.press(action_map[action])
        
        # Checking if the game is over
        done = self.get_over()
        # Get the next observation
        new_observation = self.get_observation()
        # Use score as reward
        reward_img = env.screenshot_helper.screenshot(hwnd, bbox, )
        reward = env.text_recog_helper.get_number_from_image(reward_img)
        info = {}

        return new_observation, reward, done, info

    # Visualize the game (not that important)
    def render(self):
        # cv2.imshow('Game', self.get_observation())
        # if cv2.waitKey(1) & 0xFF == ord('q')
        #     self.close()
        pass

    # Restart the game
    def reset(self):
        time.sleep(5)
        pydirectinput.press('enter')
        time.sleep(2)
        pydirectinput.press('d')
        pydirectinput.press('enter')
        return

    # Closes the render (not that important)
    def close(self):
        pass

    # Get the part of observation of the game that we want
    def get_observation(self):
        # Use Helper class to get screenshot
        stepWindow = pygetwindow.getWindowsWithTitle('StepMania')
        hwnd = stepWindow[0]._hWnd
        bbox = win32gui.GetWindowRect(hwnd)

        img = self.screenshot_helper.screenshot(hwnd, bbox, self.game_location)

        channel = self.screenshot_helper.downscaleImage(img, (100, 170), (1, 170, 100))
        # TODO: Why does this not return an image?
        return channel

    # Get if the game is over
    def get_over(self):
        # Use Helper class to get screenshot
        stepWindow = pygetwindow.getWindowsWithTitle('StepMania')
        hwnd = stepWindow[0]._hWnd
        bbox = win32gui.GetWindowRect(hwnd)

        done_capture = self.screenshot_helper.screenshot(hwnd, bbox, self.done_location)
        # Valid done text
        done_strings = ['Your Results']

        return self.text_recog_helper.is_text_in_image(done_capture, done_strings, 12)


In [14]:
env = StepManiaEnv()

In [9]:
obs = env.get_observation()

In [15]:
if env.get_over()[0] is True:
  time.sleep(4)
  env.reset()

In [ ]:
plt.imshow(np.array(cv2.cvtColor(env.get_observation()[0], cv2.COLOR_BGR2RGB)))